In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cyberbullying-classification/cyberbullying_tweets.csv


In [2]:
file_name = '/kaggle/input/cyberbullying-classification/cyberbullying_tweets.csv'
tweets = pd.read_csv(file_name)
tweets.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [3]:
import csv
import string
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')

sentences = []
labels = []
translate_table = dict((ord(char), None) for char in string.punctuation)
stop_words = set(stopwords.words('english'))

with open(file_name, encoding='UTF-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        if row[1] == 'not_cyberbullying':
            labels.append(0)
        else:
            labels.append(1)
        sentence = row[0].lower()
        sentence = sentence.replace(",", " , ")
        sentence = sentence.replace(".", " . ")
        sentence = sentence.replace("-", " - ")
        sentence = sentence.replace("/", " / ")
        sentence = sentence.translate(translate_table)
        sentence = " ".join([w for w in sentence.split() if w not in stop_words])
        sentences.append(sentence)


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
print(len(sentences))
print(len(labels))

47693
47693


In [5]:
training_size = 40000
training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

In [6]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
vocab_size = 22000
max_length = 20
oov_tok = "<OOV>"
trunc_type = "post"
padding_type = "post"

tk = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tk.fit_on_texts(training_sentences)

word_index = tk.word_index

training_sequence = tk.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequence,
                               maxlen = max_length,
                               padding = padding_type,
                               truncating = trunc_type)

2024-06-06 15:36:41.992907: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-06 15:36:41.992998: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-06 15:36:41.994709: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
print(training_padded[0])
print(training_sentences[0])

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int32)

In [11]:
print(training_padded[1])
print(training_sentences[1])

[ 330 2103  594    1   19    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0]
words katandandre food crapilicious mkr


In [12]:
print(training_padded[2])
print(training_sentences[2])

[    1    94    19 15104  8714   183 15105     1 12001     1   214     0
     0     0     0     0     0     0     0     0]
aussietv white mkr theblock imacelebrityau today sunrise studio10 neighbours wonderlandten etc
